In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.autograd import Variable
import math
import tqdm
import pdb
from scipy.sparse import csr_matrix, linalg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings

warnings.filterwarnings(action='ignore')


In [84]:
books = pd.read_csv("../data/books.csv")
train = pd.read_csv("../data/train_ratings.csv")
test = pd.read_csv("../data/test_ratings.csv")
users = pd.read_csv("../data/users.csv")
sub = pd.read_csv("../data/sample_submission.csv")
total = pd.read_csv("../data/total_ratings.csv")

In [85]:
train

,user_id,isbn,rating
0,8,0002005018,4
1,67544,0002005018,7
2,123629,0002005018,8
3,200273,0002005018,8
4,210926,0002005018,9
...,...,...,...
306790,278843,0743525493,7
306791,278851,067161746X,6
306792,278851,0884159221,7
306793,278851,0912333022,7


In [86]:
test

,user_id,isbn,rating
0,11676,0002005018,0
1,116866,0002005018,0
2,152827,0060973129,0
3,157969,0374157065,0
4,67958,0399135782,0
...,...,...,...
76694,278543,1576734218,0
76695,278563,3492223710,0
76696,278633,1896095186,0
76697,278668,8408044079,0


In [87]:
train["user_id"]

0              8
1          67544
2         123629
3         200273
4         210926
           ...  
306790    278843
306791    278851
306792    278851
306793    278851
306794    278851
Name: user_id, Length: 306795, dtype: int64

In [ ]:
train

In [46]:
df = train.merge(users, on='user_id', how='left').merge(books, on='isbn', how='left')

In [47]:
df.head()

,user_id,isbn,rating,location,age,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,8,0002005018,4,"timmins, ontario, canada",NaN,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,67544,0002005018,7,"toronto, ontario, canada",30.0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
2,123629,0002005018,8,"kingston, ontario, canada",NaN,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
3,200273,0002005018,8,"comber, ontario, canada",NaN,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
4,210926,0002005018,9,"guelph, ontario, canada",NaN,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg


In [48]:
user_id_rating_mean = df.groupby("user_id")["rating"].mean()

In [50]:
user_id_rating_mean

user_id
8          4.428571
9          5.000000
12        10.000000
14         2.500000
16         9.000000
            ...    
278846     8.000000
278849     9.000000
278851     5.833333
278852     8.000000
278854     7.000000
Name: rating, Length: 59803, dtype: float64

In [55]:
user_id_rating_mean = user_id_rating_mean.rename('mean')

In [56]:
user_id_rating_count = df.groupby("user_id")["rating"].count()

In [57]:
user_rating = pd.concat([user_id_rating_mean,user_id_rating_count],axis=1)

In [58]:
user_rating

,mean,rating
user_id,,
8,4.428571,7
9,5.000000,1
12,10.000000,1
14,2.500000,2
16,9.000000,1
...,...,...
278846,8.000000,1
278849,9.000000,1
278851,5.833333,12


In [78]:
user_rating.sort_values(['rating','mean'],ascending = [False, True]).head(50)

,mean,rating
user_id,,
11676,6.779891,5520
98391,8.566228,4560
189835,3.184298,1503
153662,8.209893,1496
23902,7.174686,956
235105,7.713054,812
76499,8.477778,810
171118,6.961089,771
16795,6.423684,760


In [68]:
user_rating.sort_values(['rating','mean'],ascending = [False, True]).tail(50)

,mean,rating
user_id,,
276045,10.0,1
276074,10.0,1
276078,10.0,1
276119,10.0,1
276142,10.0,1
276170,10.0,1
276226,10.0,1
276247,10.0,1
276253,10.0,1


In [94]:
test_userid= test.groupby('user_id').count().iloc[:,1]
test_userid = test_userid.rename('test_rating')

In [101]:
test_userid.sort_values()

user_id
14           1
164757       1
164735       1
164706       1
164667       1
          ... 
23902      223
153662     347
189835     396
98391     1125
11676     1419
Name: test_rating, Length: 26167, dtype: int64

In [100]:
test

,user_id,isbn,rating
0,11676,0002005018,0
1,116866,0002005018,0
2,152827,0060973129,0
3,157969,0374157065,0
4,67958,0399135782,0
...,...,...,...
76694,278543,1576734218,0
76695,278563,3492223710,0
76696,278633,1896095186,0
76697,278668,8408044079,0


In [88]:
train.groupby(["user_id"]).mean()

,rating
user_id,
8,4.428571
9,5.000000
12,10.000000
14,2.500000
16,9.000000
...,...
278846,8.000000
278849,9.000000
278851,5.833333


In [109]:
train.nunique()

user_id     59803
isbn       129777
rating         10
dtype: int64

In [110]:
#train user_id별 rating 개수
series=train.groupby(["user_id"])["rating"].count()==1
series.value_counts()

True     34617
False    25186
Name: rating, dtype: int64

In [126]:
print(f"train데이터에서 유저:",train['user_id'].nunique(),"명")
print("train데이터에서 1개 평점 남긴 사람 비율:",34617/train['user_id'].nunique())
print("train데이터에서 1개 평점 남긴 사람 비율:",25186/train['user_id'].nunique())

train데이터에서 유저: 59803 명
train데이터에서 1개 평점 남긴 사람 비율: 0.5788505593364881
train데이터에서 1개 평점 남긴 사람 비율: 0.42114944066351184


In [127]:
pd.concat([train.groupby(["user_id"])["rating"].count(),test.groupby(["user_id"])["rating"].count()], axis = 1).info()

user_id
8          7
9          1
12         1
14         2
16         1
          ..
278846     1
278849     1
278851    12
278852     1
278854     3
Name: rating, Length: 59803, dtype: int64

In [138]:
test["user_id"].nunique()

26167

In [134]:
print("test가 train에 있는 유저",59803 - 26167, "비율:",(59803 - 26167)/train['user_id'].nunique())
print("test가 train에 없는 새로운 유저",26167, "비율:",(26167)/train['user_id'].nunique())

test가 train에 있는 유저 33636 비율: 0.5624466999983279
test가 train에 없는 새로운 유저 26167 비율: 0.43755330000167214


In [136]:
pd.concat([train.groupby(["user_id"])["rating"].count(),test.groupby(["user_id"])["rating"].count()], axis = 1)

,rating,rating
user_id,,
8,7.0,NaN
9,1.0,NaN
12,1.0,NaN
14,2.0,1.0
16,1.0,NaN
...,...,...
278767,NaN,1.0
278798,NaN,2.0
278820,NaN,1.0


In [127]:
pd.concat([train.groupby(["user_id"])["rating"].count(),test.groupby(["user_id"])["rating"].count()], axis = 1).info()

user_id
8          7
9          1
12         1
14         2
16         1
          ..
278846     1
278849     1
278851    12
278852     1
278854     3
Name: rating, Length: 59803, dtype: int64

In [95]:
usercheck = pd.concat([user_rating,test_userid],axis=1)

In [96]:
usercheck

,mean,rating,test_rating
user_id,,,
8,4.428571,7.0,NaN
9,5.000000,1.0,NaN
12,10.000000,1.0,NaN
14,2.500000,2.0,1.0
16,9.000000,1.0,NaN
...,...,...,...
278767,NaN,NaN,1.0
278798,NaN,NaN,2.0
278820,NaN,NaN,1.0


In [103]:
68069 - 26167

41902

In [104]:

# 새로운 user : 8266
# 평가 기록은 있는 user : 41902
usercheck.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68069 entries, 8 to 278836
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   mean         59803 non-null  float64
 1   rating       59803 non-null  float64
 2   test_rating  26167 non-null  float64
dtypes: float64(3)
memory usage: 2.1 MB


In [13]:
df

,user_id,isbn,rating,location,age,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,8,0002005018,4,"timmins, ontario, canada",NaN,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,67544,0002005018,7,"toronto, ontario, canada",30.0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
2,123629,0002005018,8,"kingston, ontario, canada",NaN,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
3,200273,0002005018,8,"comber, ontario, canada",NaN,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
4,210926,0002005018,9,"guelph, ontario, canada",NaN,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306790,278843,0743525493,7,"pismo beach, california, usa",28.0,The Motley Fool's What To Do with Your Money N...,David Gardner,2002.0,Simon & Schuster Audio,http://images.amazon.com/images/P/0743525493.0...,NaN,NaN,NaN,images/0743525493.01.THUMBZZZ.jpg
306791,278851,067161746X,6,"dallas, texas, usa",33.0,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987.0,Pocket Books,http://images.amazon.com/images/P/067161746X.0...,en,['Humor'],A tongue-in-cheek survival guide for single pe...,images/067161746X.01.THUMBZZZ.jpg
306792,278851,0884159221,7,"dallas, texas, usa",33.0,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985.0,Lone Star Books,http://images.amazon.com/images/P/0884159221.0...,NaN,NaN,NaN,images/0884159221.01.THUMBZZZ.jpg
306793,278851,0912333022,7,"dallas, texas, usa",33.0,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997.0,Kqed Books,http://images.amazon.com/images/P/0912333022.0...,en,['Fiction'],These hilarious stories by the creator of publ...,images/0912333022.01.THUMBZZZ.jpg


In [18]:
df['summary'].values

array(['In a small town in Canada, Clara Callan reluctantly takes leave of her\nsister, Nora, who is bound for New York.',
       'In a small town in Canada, Clara Callan reluctantly takes leave of her\nsister, Nora, who is bound for New York.',
       'In a small town in Canada, Clara Callan reluctantly takes leave of her\nsister, Nora, who is bound for New York.',
       ..., nan,
       'These hilarious stories by the creator of public television&#39;s\nlongest-running hit series capture the wacky sensibility and off-the-\nwall humor of the British sitcom.',
       nan], dtype=object)

In [28]:
total

,Unnamed: 0,user_id,isbn,rating
0,0,8,0771074670,1
1,1,8,0002005018,5
2,2,17,0684823802,1
3,3,17,0425099148,7
4,4,17,0553264990,5
...,...,...,...,...
164724,164724,278854,0553578596,1
164725,164725,278854,0316184152,7
164726,164726,278854,0515087122,1
164727,164727,278854,0553579606,8


In [50]:
testset= pd.concat([total, test],axis=1)

In [51]:
testset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164729 entries, 0 to 164728
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  164729 non-null  int64  
 1   user_id     164729 non-null  int64  
 2   isbn        164729 non-null  object 
 3   rating      164729 non-null  int64  
 4   user_id     76699 non-null   float64
 5   isbn        76699 non-null   object 
 6   rating      76699 non-null   float64
dtypes: float64(2), int64(3), object(2)
memory usage: 8.8+ MB


In [52]:
df = test.merge(total, on=['user_id','isbn'], how='left')

In [53]:
df

,user_id,isbn,rating_x,Unnamed: 0,rating_y
0,11676,0002005018,0,8532.0,8.0
1,116866,0002005018,0,71340.0,9.0
2,152827,0060973129,0,NaN,NaN
3,157969,0374157065,0,NaN,NaN
4,67958,0399135782,0,NaN,NaN
...,...,...,...,...,...
76694,278543,1576734218,0,NaN,NaN
76695,278563,3492223710,0,NaN,NaN
76696,278633,1896095186,0,NaN,NaN
76697,278668,8408044079,0,NaN,NaN


In [54]:
df = df[df['rating_y'].notnull()]
del df["Unnamed: 0"]

In [55]:
df.rename(columns={"rating_y":"rating"},inplace=True)

In [58]:
del df["rating_x"]

In [62]:
df.to_csv("testcheat.csv")

In [61]:
df

,user_id,isbn,rating
0,11676,0002005018,8.0
1,116866,0002005018,9.0
10,7352,0440234743,8.0
12,13552,0440234743,8.0
15,64535,0440234743,7.0
...,...,...,...
71332,184985,0140445927,9.0
71361,242006,0671723421,8.0
71373,209464,0141310464,7.0
71544,260699,1567184464,10.0


In [63]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76699 entries, 0 to 76698
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  76699 non-null  int64 
 1   isbn     76699 non-null  object
 2   rating   76699 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.8+ MB


In [64]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306795 entries, 0 to 306794
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  306795 non-null  int64 
 1   isbn     306795 non-null  object
 2   rating   306795 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 7.0+ MB


In [65]:
df1 = train.merge(total, on=['user_id','isbn'], how='left')

In [66]:
#1
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306795 entries, 0 to 306794
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     306795 non-null  int64  
 1   isbn        306795 non-null  object 
 2   rating_x    306795 non-null  int64  
 3   Unnamed: 0  101513 non-null  float64
 4   rating_y    101513 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 14.0+ MB


In [68]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306795 entries, 0 to 306794
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  306795 non-null  int64 
 1   isbn     306795 non-null  object
 2   rating   306795 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 7.0+ MB


In [69]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76699 entries, 0 to 76698
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  76699 non-null  int64 
 1   isbn     76699 non-null  object
 2   rating   76699 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.8+ MB


In [70]:
76699/306795

0.25000081487638326